## Unsupervised Keyword Extraction: keyBERT

Author: Sumaia Parveen Shupti

Created on: 07/14/2021

Updated on: 08/02/2021

In [1]:
!pip install wikipedia
!pip install rake-nltk
!pip install keybert
!pip install flair
!pip install keybert[gensim]
!pip install keybert[spacy]
!pip install keybert[use]
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=e98faa4755c994fa95e79ba2099071a7ed35e78a2d81a62ce7e4254e920a6d83
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7829 sha256=d107e0d31655c3fa26c718e6f91be947551fea0baa5d5c9d2d784debe77481df
  Stored in directory: /root/.cache/pip/wheels/7c/d9/8a/b8a9244fa89a07f288f9fe006aafc79d93fceb58496c29b606
Successfully built rake-nltk
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 208 kB 26.7 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 
     |████████████████████████████████| 2.6 MB 45.7 MB/s 
     |████████████████████████████████| 1.2 MB 45.5 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 895 kB 60.3 MB/s 
     |███████

In [2]:
import wikipedia
import pandas as pd
 
# Get wiki content.
wikisearch = wikipedia.page("UFO")
wikicontent = wikisearch.content
wikicontent

'An unidentified flying object (UFO) is any perceived aerial phenomenon that cannot immediately be identified or explained. Most UFOs are identified or investigated as conventional objects or phenomena. The phrase was coined as an acronym by Project Blue Book project head Edward J. Ruppelt, but today UFO is widely used as a stand-in for extraterrestrial spacecraft-aircraft claimed to be observed by various people. Many UFOs are described as being flying saucers, as is shown in the image to the right.\nStudies and investigations into UFO reports have been conducted by various governments worldwide, along with private individuals and organizations. In the United States, studies began in the late 1940s and have included Project Grudge, Project Sign and Project Blue Book. The latter was ended in 1969–1970 after the Condon Committee officially concluded that the subject failed to merit further study.\nUnidentified lights and flying objects have been reported in the skies for much of human h

### Intial Classification

1. Max Sum Similarity:  2 x top_n most similar words/phrases taken to the document. Then, all top_n combinations from the 2 x top_n words are taken and the combination that are the least similar to each other by cosine similarity are extracted.

2. Maximal Marginal Relevance: Maximal Margin Relevance (MMR) can be used to create keywords / keyphrases which is also based on cosine similarity. Users input the percentage of diversity into the model.

### Max Sum Similarity (Sentence Transformer)

In [3]:
def sent_trans_maxsum(raw_text, n_gram, no_kw, model_name):
    
    from keybert import KeyBERT
    from sentence_transformers import SentenceTransformer

    model_ = SentenceTransformer(model_name)
    kw_model = KeyBERT(model=model_)

    kw = []
    weight = []

    a = kw_model.extract_keywords(raw_text, keyphrase_ngram_range=(int(n_gram), int(n_gram)), stop_words='english', 
                              use_maxsum=True, nr_candidates=int(no_kw), top_n=int(no_kw))

    for i in list(range(len(a))):
            kw.append(a[i][0])
            weight.append(a[i][1])

    df = pd.DataFrame()    
    df['Kewrord'] = kw
    df['Weight'] = weight
    df = df.sort_values(by ='Weight', ascending = False)
    df = df.head(int(no_kw))       
    df = df.reset_index(drop = True)

    return df  

In [4]:
%time sent_trans_maxsum(wikicontent, 3, 25, 'paraphrase-mpnet-base-v2')


CPU times: user 3min, sys: 6.69 s, total: 3min 7s
Wall time: 3min 14s


,Kewrord,Weight
0,defined unidentified flying,0.5056
1,dictionary defines ufo,0.4117
2,defines ufo unidentified,0.3840
3,flying object ufo,0.2578
4,ufo ufob coined,0.2209
5,terminology term ufo,0.2080
6,term ufo generally,0.1966
7,technically ufo refers,0.1957
8,acronym ufo coined,0.1800
9,usage term ufo,0.1574


###  Maximal Marginal Relevance (Sentence Transformer)

In [5]:
def sent_trans_mmr(raw_text, n_gram, no_kw, model_name):
    
    from keybert import KeyBERT
    from sentence_transformers import SentenceTransformer

    model_ = SentenceTransformer(model_name)
    kw_model = KeyBERT(model=model_)

    kw = []
    weight = []

    a = kw_model.extract_keywords(raw_text, keyphrase_ngram_range=(int(n_gram), int(n_gram)), stop_words='english', 
                              use_mmr=True, diversity=0.7, nr_candidates=int(no_kw), top_n=int(no_kw))

    for i in list(range(len(a))):
            kw.append(a[i][0])
            weight.append(a[i][1])

    df = pd.DataFrame()    
    df['Kewrord'] = kw
    df['Weight'] = weight
    df = df.sort_values(by ='Weight', ascending = False)
    df = df.head(int(no_kw))       
    df = df.reset_index(drop = True)

    return df 

In [6]:
%time sent_trans_mmr(wikicontent, 3, 25, 'paraphrase-mpnet-base-v2')

CPU times: user 2min 45s, sys: 2.67 s, total: 2min 48s
Wall time: 2min 46s


,Kewrord,Weight
0,defines ufo unidentified,0.8480
1,mythology cultism associated,0.2663
2,unknowns involved blue,0.2523
3,english dictionary defines,0.2360
4,fbi cia documents,0.2034
5,mexico lapaz reported,0.2032
6,japan air lines,0.1995
7,elmo planet venus,0.1752
8,consciousness reported having,0.1736
9,report uaptf unable,0.1688


### Further Classification: Embedding Models

1. Sentence-Transformers
2. Flair
3. Spacy
4. Gensim
5. USE

** We will work with Max Sum Similarity here.

### Sentence Transformers

Any of the pretrained sentence embedding models can be used from here: https://www.sbert.net/docs/pretrained_models.html

In [7]:
%time sent_trans_maxsum(wikicontent, 1, 10, 'paraphrase-MiniLM-L6-v2') 


CPU times: user 9.58 s, sys: 951 ms, total: 10.5 s
Wall time: 14.5 s


,Kewrord,Weight
0,ufobs,0.1658
1,extraterrestrials,0.0708
2,extraterrestrial,0.0313
3,ufologist,0.0307
4,ufob,0.0297
5,ufology,0.0204
6,ufologists,0.0124
7,aliens,0.0109
8,ufos,0.0037
9,ufo,-0.0116


### Flair

Any of the pretrained embedding models can be used from here: https://huggingface.co/models

In [8]:
def flair_maxsum(raw_text, n_gram, no_kw):

    from keybert import KeyBERT
    from flair.embeddings import TransformerDocumentEmbeddings

    roberta = TransformerDocumentEmbeddings('roberta-base')
    kw_model = KeyBERT(model=roberta)

    kw = []
    weight = []

    a = kw_model.extract_keywords(raw_text, keyphrase_ngram_range=(int(n_gram), int(n_gram)), stop_words='english', 
                              use_maxsum=True, nr_candidates=int(no_kw), top_n=int(no_kw))

    for i in list(range(len(a))):
            kw.append(a[i][0])
            weight.append(a[i][1])

    df = pd.DataFrame()    
    df['Kewrord'] = kw
    df['Weight'] = weight
    df = df.sort_values(by ='Weight', ascending = False)
    df = df.head(int(no_kw))       
    df = df.reset_index(drop = True)

    return df  

In [9]:
%time flair_maxsum(wikicontent, 1, 10) 

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 2min 52s, sys: 5.19 s, total: 2min 57s
Wall time: 3min 1s


,Kewrord,Weight
0,disestablished,0.9924
1,skeptics,0.9922
2,beliefs,0.9922
3,unidentifiable,0.9921
4,earthbound,0.9920
5,historians,0.9920
6,satellites,0.9919
7,skeptical,0.9919
8,controversy,0.9915
9,subjects,0.9914


### USE

In [10]:
def use(raw_text, n_gram, no_kw):
    

    from keybert import KeyBERT
    import tensorflow_hub

    model_ = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    kw_model = KeyBERT(model=model_)

    kw = []
    weight = []

    a = kw_model.extract_keywords(raw_text, keyphrase_ngram_range=(int(n_gram), int(n_gram)), stop_words='english', 
                              use_maxsum=True, nr_candidates=int(no_kw), top_n=int(no_kw))

    for i in list(range(len(a))):
            kw.append(a[i][0])
            weight.append(a[i][1])
            

    df = pd.DataFrame()    
    df['Kewrord'] = kw
    df['Weight'] = weight
    df = df.sort_values(by ='Weight', ascending = False)
    df = df.head(int(no_kw))       
    df = df.reset_index(drop = True)

    return df 

In [11]:
%time use(wikicontent, 1, 10)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 880.00MB
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


Instructions for updating:
Use tf.identity instead.


Instructions for updating:
Use tf.identity instead.


CPU times: user 22.5 s, sys: 5.74 s, total: 28.3 s
Wall time: 29.7 s


,Kewrord,Weight
0,sightings,0.0827
1,abduction,0.0671
2,aliens,0.0668
3,ufo,0.0651
4,extraterrestrial,0.0055
5,roswell,-0.0109
6,unidentified,-0.0221
7,extraterrestrials,-0.0364
8,saucers,-0.0510
9,ufos,-0.0893


### Gensim

In [12]:
def gensim(raw_text, n_gram, no_kw):
    
    import gensim.downloader as api
    from keybert import KeyBERT
    ft = api.load('fasttext-wiki-news-subwords-300')
    kw_model = KeyBERT(model=ft)

    kw = []
    weight = []

    a = kw_model.extract_keywords(raw_text, keyphrase_ngram_range=(int(n_gram), int(n_gram)), stop_words='english', 
                              use_maxsum=True, nr_candidates=int(no_kw), top_n=int(no_kw))

    for i in list(range(len(a))):
            kw.append(a[i][0])
            weight.append(a[i][1])
            
    df = pd.DataFrame()    
    df['Kewrord'] = kw
    df['Weight'] = weight
    df = df.sort_values(by ='Weight', ascending = False)
    df = df.head(int(no_kw))       
    df = df.reset_index(drop = True)

    return df

In [13]:
%time gensim(wikicontent, 1, 10)

[==================================================] 100.0% 958.5/958.4MB downloaded
CPU times: user 9min 51s, sys: 41.9 s, total: 10min 33s
Wall time: 11min 58s


,Kewrord,Weight
0,various,0.4011
1,relevant,0.3869
2,work,0.3394
3,actual,0.3362
4,present,0.3337
5,just,0.3307
6,thought,0.3279
7,supposed,0.3063
8,main,0.2268
9,given,0.2238


### References

* https://github.com/MaartenGr/KeyBERT
* https://maartengr.github.io/KeyBERT/guides/embeddings.html